# Image Embedding

In [ ]:
from PIL import Image
import numpy as np

In [ ]:
img=Image.open("miku.jpg").convert("RGB")
img

In [ ]:
# img=Image.open("lena_gray.gif").convert("L")
# img

In [ ]:
img.mode

In [ ]:
nparr=np.array(img)
nparr

In [ ]:
nparr.shape

In [ ]:
def getblocks(image: np.ndarray, blockshape: tuple, moveAxis: bool = True) -> np.ndarray:
    '''
    takes the array of image in grey= 2D and in RGB = 3D
    takes the numpy array and converts it the the blocks in the fastest way
    '''
    if len(image.shape) == 2:
        mode = "grey"
        image = image.reshape(*image.shape, 1)
    else:
        mode = "color"

    img_height, img_width, channels = image.shape
    tile_height, tile_width = blockshape

    if img_height % tile_height != 0 or img_width % tile_width != 0:
        print("warning: Block size is not fit for the image!")

    tiled_array = image.reshape(
        img_height//tile_height, tile_height, img_width//tile_width, tile_width, channels)
    tiled_array = tiled_array.swapaxes(1, 2)

    if moveAxis:
        tiled_array = tiled_array.reshape(-1,
                                          *(tile_height, tile_width, channels))
        tiled_array = np.moveaxis(tiled_array, source=len(
            tiled_array.shape)-1, destination=1)

    return tiled_array


def combineBlocks(tiled_array: np.ndarray, imageshape: tuple, blockshape: tuple, movedAxis: bool = True) -> np.ndarray:

    if len(imageshape) == 2:
        mode = "grey"
        imageshape = *imageshape, 1
    else:
        mode = "color"

    img_height, img_width, channels = imageshape
    tile_height, tile_width = blockshape

    if movedAxis:
        image = tiled_array.copy()
        image = image.reshape(img_height//tile_height, tile_height,
                              img_width//tile_width, tile_width, channels)
        swapaxisShape = list(image.shape)
        swapaxisShape[1], swapaxisShape[2] = swapaxisShape[2], swapaxisShape[1]
        image = image.reshape(swapaxisShape)
        image = image.swapaxes(1, 2)
    else:
        image = tiled_array
        # IDK have completed this else case. Btw we aren't using this case lol :)

    return image.reshape(imageshape)


In [ ]:
%timeit getblocks(nparr,(2,2), True)

In [ ]:
blocksize=(4,2)

In [ ]:
blocks=getblocks(nparr, blocksize, True)
blocks

In [ ]:
blocks.shape


In [ ]:
blocks[0][0]


## PSNR

In [ ]:
from math import log10, sqrt

In [ ]:
def PSNR(original, compressed):
    mse = np.mean((original - compressed) ** 2)
    if(mse == 0):  # MSE is zero means no noise is present in the signal .
                  # Therefore PSNR have no importance.
        return 100
    max_pixel = 255.0
    psnr = 20 * log10(max_pixel / sqrt(mse))
    return psnr

In [ ]:
# print(f"PSNR value is {value} dB")
PSNR(nparr, nparr)

In [ ]:
avgblock=blocks.copy()

In [ ]:
avgblock[0][0]

In [ ]:
avgblock[0][0]

In [ ]:
# for i in range(avgblock.shape[0]):
#     avgblock[i][0][:] = int(avgblock[i][0].mean())
#     avgblock[i][1][:] = int(avgblock[i][1].mean())
#     avgblock[i][2][:] = int(avgblock[i][2].mean())


In [ ]:
# avgblock[0][0][0]=69
# avgblock[2][0][0]=69

In [ ]:
avgimage=combineBlocks(avgblock, nparr.shape, blocksize)

In [ ]:
avgimage.shape

In [ ]:
avgimage

In [ ]:
(avgimage.reshape(-1)==nparr.reshape(-1)).all()

In [ ]:
Image.fromarray(avgimage, 'RGB')

In [ ]:
PSNR(nparr,avgimage)

29.810711022024247 psnr for 2x4
29.83055903916751 psnr for 4x2

In [ ]:
avgimage

for the 4x2 PSNR

In [ ]:
blocksize=(4,2)

In [ ]:
blocks = nparr.reshape(
    (int(imagedim[0]/blocksize[0]), int(imagedim[1]/blocksize[1]), blocksize[0], blocksize[1]))
np.set_printoptions(threshold=np.inf)
print(nparr.reshape(-1))

In [ ]:
blocks.shape


In [ ]:
avgblock=blocks.copy()

In [ ]:
for i in range(avgblock.shape[0]):
    for j in range(avgblock.shape[1]):
        avgblock[i][j][:] = int(avgblock[i][j].mean())


In [ ]:
avgblock[0][0]

In [ ]:
image2=avgblock.reshape((512,512))
image2

In [ ]:
PSNR(nparr,image2)

In [ ]:
Image.fromarray(image2, 'L')